<a href="https://colab.research.google.com/github/anmolag190153/BCS_summer_project_SER/blob/main/Speech_emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SPEECH EMOTION RECOGNITION PROJECT
## BCS-IITK


# Emotions feature extraction

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# path1='/content/drive/My Drive/Ravdess/Actor_01/03-01-08-02-02-02-01.wav'
# ipd.Audio(path1)

In [3]:
# actor_1,sr=librosa.load(path1)
# print(actor_1.shape)
# print(sr)
sr=22100

In [4]:
# plt.figure(figsize=(15,5))
# librosa.display.waveplot(actor_1,sr=sr)
# plt.ylim((-0.1,0.1))
# plt.show()

In [5]:
def extract_feature(file):
    result = np.array([])
    mfcc = np.mean(librosa.feature.mfcc(file,sr=sr,n_mfcc=20).T,axis=0)
    result = np.hstack((result,mfcc))
    # chroma=np.mean(librosa.feature.chroma_stft(file,sr=sr).T,axis=0)
    # result=np.hstack((result,chroma))
    # mel = np.mean(librosa.feature.melspectrogram(file,sr=sr).T,axis=0)
    # result = np.hstack((result,mel))
    # spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(file, sr=sr).T,axis=0)
    # result = np.hstack((result,spectral_rolloff))
    spec_centroid = np.mean(librosa.feature.spectral_centroid(file,sr).T,axis=0)
    result = np.hstack((result,spec_centroid))
    spec_bandwidth = np.mean(librosa.feature.spectral_bandwidth(file,sr).T,axis=0)
    result = np.hstack((result,spec_bandwidth))
    spec_contrast = np.mean(librosa.feature.spectral_contrast(file,sr).T,axis=0)
    result=np.hstack((result,spec_contrast))
    spec_flat = np.mean(librosa.feature.spectral_flatness(file).T,axis=0)
    result=np.hstack((result,spec_flat))
    return result

# Loading Data
We are using the RAVDESS dataset for this project.

In [6]:
path='../BCS_summer_project_SER/Audio Datasets'
k=0
x_data=np.ndarray(shape=(24*60,120000))
y_data=np.ndarray(shape=(24*60,1))
for i,actor in enumerate(os.listdir(path)):
    path1=os.path.join(path,actor)
    for j,voice in enumerate(os.listdir(path1)):
        if voice=='test':
            continue
        b=voice[6:8]
        y_data[k,0]=b
        y_data[k,0]=y_data[k,0]-1
        temp_path=os.path.join(path1,voice)
        # print(temp_path)
        var,_=librosa.load(temp_path)
        x_data[k,:var.shape[0]]=var[:]
        x_data[k,var.shape[0]:]=0
        k=k+1

# Extracting Features
We have used mfccs, chroma frequency, mel spectogram and spectral centroid as audio features in this project.

In [7]:
x=[]
for i in range(24*60):
    feature=extract_feature(x_data[i,:])
    x.append(feature)

**Scaling** the data and splitting data for **training** and **validation**

In [8]:
x_train, x_test,y_train,y_test = train_test_split(np.array(x), y_data, test_size=0.2, random_state=9)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Saving features 

In [9]:
for i,feat in enumerate(x_train):
    np.savetxt('features/train/'+str(i)+'_'+str(y_train[i])+'.csv',feat,delimiter=',')
for i,feat in enumerate(x_test):
    np.savetxt('features/test/'+str(i)+'_'+str(y_test[i])+'.csv',feat,delimiter=',')

In [10]:
print((x_train.shape,x_test.shape,y_train.shape))

((1152, 30), (288, 30), (1152, 1))
